In [31]:
# ! pip install autokeras

In [32]:
# ! pip list

In [33]:
import pandas as pd
import numpy as np
import itertools
# import matplotlib.pyplot as plt
import useful_functions as uf
# from sklearn.preprocessing import OneHotEncoder
path_fill = '/workspaces/CMB_match/data/2022/filled_trian.csv'
path_train_pure = '/workspaces/CMB_match/data/2022/train.csv'
# data/2022/filled_trian.csv
df_train_filled = pd.read_csv(path_fill, index_col=0)
df_train = pd.read_csv(path_train_pure)

有序型分类变量：使用LabelEncoder()编码；

无序型分类变量：使用独热编码。

In [34]:
string_col =uf.dynamic_string_col(df_train_filled)

opt_col = string_col.copy()

df_str=pd.DataFrame()
if 'MON_12_CUST_CNT_PTY_ID' in string_col:
    # This a Binary Y/N, the training sample has Y only, do not fill with the mode
    opt_col.remove('MON_12_CUST_CNT_PTY_ID')
    df_str.loc[:,'MON_12_CUST_CNT_PTY_ID'] = df_train_filled['MON_12_CUST_CNT_PTY_ID']
    # Direct copy
# fill the missing value with mode
df_str.loc[:,opt_col] = uf.mode_nan_string(df_train_filled.loc[:, opt_col])
## One-hot encoding
df_str.loc[:,'SHH_BCK']=df_str.SHH_BCK.astype(str) # it was floated
dummies = pd.get_dummies(df_str,dummy_na=True)

In [35]:
_df_numerical= uf.get_numerical_df(df_train_filled)

1 无序分类变量的相关性
2 get the correlation matrix of the numerical variables
3 save the first column with the lowest missing value

In [36]:
corr_cols = uf.high_corr_col(df_train_filled)
drop_col = uf.save_n_drop(corr_cols,df_train_filled)
test = df_train_filled.drop(drop_col,axis=1)
second_corr = test.loc[:,uf.high_corr_col(test)].corr()
second_corr

,MON_12_EXT_SAM_TRSF_IN_AMT,EMP_NBR,LAST_12_MON_COR_DPS_DAY_AVG_BAL,CUR_MON_COR_DPS_MON_DAY_AVG_BAL,HLD_DMS_CCY_ACT_NBR
MON_12_EXT_SAM_TRSF_IN_AMT,1.000000,0.007209,0.448644,0.415210,0.055374
EMP_NBR,0.007209,1.000000,0.014451,0.089909,0.802149
LAST_12_MON_COR_DPS_DAY_AVG_BAL,0.448644,0.014451,1.000000,0.935051,0.087795
CUR_MON_COR_DPS_MON_DAY_AVG_BAL,0.415210,0.089909,0.935051,1.000000,0.143070
HLD_DMS_CCY_ACT_NBR,0.055374,0.802149,0.087795,0.143070,1.000000


There are still four columns i.e. two paris are high correlated variables

In [37]:
rows,cols = np.where(second_corr>0.8)
# gives the position of all correlation value >0.8
get_rid_2=list()
for i in range(len(rows)):
    posi= second_corr.iloc[rows[i],cols[i]]
    # rows[i],cols[i] is the position of the element in the matrix
    # posi is the correlation value of rows[i] and cols[i]
    if posi<0.9990: # type: ignore
        col_set = {second_corr.index[rows[i]], second_corr.columns[cols[i]]}
        # this is the actual column name
        if col_set not in get_rid_2: # because the set disregard the order,
            # so we need to check if the set is already in the list,
            # the pair with the same two elements
            # but different order will be regarded as same set,
            get_rid_2.append(col_set)
drop_col_2  = list(itertools.chain(*get_rid_2)) # take component out to list
# drop_col_2 is the list of column name that need to be dropped

In [39]:
get_rid_2

[{'EMP_NBR', 'HLD_DMS_CCY_ACT_NBR'},
 {'CUR_MON_COR_DPS_MON_DAY_AVG_BAL', 'LAST_12_MON_COR_DPS_DAY_AVG_BAL'}]

4 use the drop_col we get to drop again

In [38]:
rank_df = pd.DataFrame(df_train.loc[:, drop_col_2].isna().sum())
    # for each abbreviation we keep 1, we assume it is the same thing
rank_df.sort_values([0], ascending=True)
    # for each group of abbreviation, we keep only the fullest data set
# [col for col in df_train[drop_col_2].columns if col not in rank_res_col]